# 学習したモデルによる予測
---
### 利用バージョン
python 2 (python 3にしたかったけど、色々環境設定しないといけなさそうなので・・)

## 概要
ここでは、
* リサイズする解像度のサイズ、予測させたい画像ファイルのパス指定
* 予測結果の確認

といった事をします。

## 変数定義
* リサイズする解像度をIMG_SIZEにて指定。これは学習時に使用したサイズと合わせる必要があります。
* 予測したい画像をFILE_NAMEで指定します。

In [1]:
IMG_SIZE = 250
unit_size = 200
FILE_NAME = u'IMG_4071.JPG'

## 予測スタート
### 画像の読み込み&リサイズ
学習時同様、解像度を統一する必要があるため、画像ファイルを取り扱うのに便利なPILをインポートしリサイズします。あとで変換しやすいので、PILのimageオブジェクトとしてpictFile変数を保持します。

In [2]:
import os
from PIL import Image

def resizePict(filePathName):

    # 画像ファイルとして開く&リサイズできるか試す
    pictFile = Image.open(filePathName)
    reSizePictFile = pictFile.resize((IMG_SIZE, IMG_SIZE))
 
    # RGB形式ではない場合、RGB形式に変換する。
    if reSizePictFile.mode != 'RGB':
        reSizePictFile = reSizePictFile.convert('RGB')
    
    return reSizePictFile


filePathName = os.path.join(u'dataset', u'04_samplePict', FILE_NAME)
pictFile = resizePict(filePathName)

### 画像データの並び順をChainer形式に変換
PILのimageオブジェクトの並び順は(color, x, y)です。一方、Chainerは(x, y, color)の並び順にする必要があるため、transposeで変換します。また、学習時同様、画素値を 255で割り、0〜1の値に変換します。

In [3]:
import numpy as np

#PILのimageオブジェクトの並び順確認
np.asanyarray(pictFile).shape

(250, 250, 3)

In [4]:
import numpy as np

# PILは(color, x, y)、Chainerは(x, y, color)の並び順にする必要があるため、transposeで変換
# 画素値を255で割り、0〜1の値にする
x = np.asarray(pictFile).transpose(2, 0, 1).astype(np.float32)/255.
x.shape

(3, 250, 250)

### ミニバッチ形式にデータを微調整
学習したモデルはミニバッチ形式なので、データセットとしては(file, color, x , y)といった形式で保持していました。予測時も形式を合わせる必要があるため、一次元増やします。

In [5]:
# ミニバッチの形に変換
x = x[None, ...]
x.shape

(1, 3, 250, 250)

### モデル読み込み
学習時に作成したモデルを読み込みます。なお、学習時とモデルは合わせる必要があります。

In [6]:
import chainer.links as L
import chainer
from chainer import serializers

class MLP(chainer.Chain):

    def __init__(self, n_mid_units=unit_size, n_out=2):
        # パラメータを持つ層の登録
        super(MLP, self).__init__(
            l1=L.Linear(None, n_mid_units),
            l2=L.Linear(None, n_mid_units),
            l3=L.Linear(None, n_mid_units),
            l4=L.Linear(None, n_mid_units),
            l5=L.Linear(None, n_out),
        )

    def __call__(self, x):
        # データを受け取った際のforward計算を書く
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        h3 = F.relu(self.l3(h2))
        h4 = F.relu(self.l4(h3))
        return self.l5(h4)

# モデルの定義
model = MLP()
model = L.Classifier(model)

#モデルファイル読み込み
serializers.load_npz(u'result/sakamotsu.model', model)

### AIによる予測
データセットをmode.predictor()に渡して結果を予測させます。L.Classifierを使う場合はpredictor()が必要で、そうでない場合は不要のよう。

In [7]:
import chainer
import chainer.functions as F

t = model.predictor(x)
y = F.softmax(t)
ans = y.data.argmax(axis=1)[0]
print (u'予測値:{}'.format(ans))
print (u'好き(1)の可能性　:{:.2%}'.format(y.data[0][1]))
print (u'嫌い(0)の可能性　:{:.2%}'.format(y.data[0][0]))
    

予測値:1
好き(1)の可能性　:55.83%
嫌い(0)の可能性　:44.17%
